# Introduction

This report analyzes wine dataset available to the public through \href{http://archive.ics.uci.edu/ml/index.php}{UCI Machine Learning Repository} which is a collection of databases, domain theories, and data generators that are used by the machine learning community for the empirical analysis of machine learning algorithms.
The dataset can be found here: \href{http://archive.ics.uci.edu/ml/datasets/Wine}{Wine Data}.


**State of art**

This dataset is the result of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines.


Dataset consists of 178 Italian wine samples. Each sample has 14 features: one is the cultivar's identifier, and the others are chemical attributes. The cultivar is a categorical variable that can take only 3 values: (Region) 1, (Region) 2, and (Region) 3. The chemical attributes are all numerical and continuous. This is a list of the 14 variables:

Cultivar's Identifier:- 
<br/> Class Id

The Attributes are:-
*  Alcohol
*  Malic acid
*  Ash
*  Alcalinity of ash
* Magnesium
* Total phenols
* Flavanoids
* Nonflavanoid phenols
* Proanthocyanins
* Color intensity
* Hue
* OD280/OD315 of diluted wines
* Proline

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#plot libaries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA#, FastICA

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


# data mining libaries
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA#, FastICA
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, learning_curve
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE

SEED = 10 # specify seed for reproducable results

/kaggle/input/wine-data/wine.data


Using TensorFlow backend.


In [2]:
RANDOM_FOREST_PARAMS = {
    'clf__max_depth': [25, 50, 75],
    'clf__max_features': ["sqrt"], # just sqrt is used because values of log2 and sqrt are very similar for our number of features (10-19) 
    'clf__criterion': ['gini', 'entropy'],
    'clf__n_estimators': [100, 300, 500, 1000]
}

DECISION_TREE_PARAMS = {
    'clf__max_depth': [25, 50, 75],
    'clf__max_features': ["sqrt"], # just sqrt is used because values of log2 and sqrt are very similar for our number of features (10-19)
    'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_split': [6, 10, 14],
}

LOGISTIC_REGRESSION_PARAMS = {
    'clf__solver': ['liblinear'],
    'clf__C': [0.1, 1, 10],
    'clf__penalty': ['l2', 'l1']
}

KNN_PARAMS = {
    'clf__n_neighbors': [5, 15, 25, 35, 45, 55, 65],
    'clf__weights': ['uniform', 'distance'],
    'clf__p': [1, 2, 10]
}

KNN_PARAMS_UNIFORM = {
    'clf__n_neighbors': [5, 15, 25, 35, 45, 55, 65],
    'clf__weights': ['uniform'],
    'clf__p': [1, 2, 10]
}

SVM_PARAMS = [
{
    'clf__kernel': ['linear'],
    'clf__C': [0.1, 1, 10],
}, 
{
    'clf__kernel': ['rbf'],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}]

In [3]:
data=pd.read_csv('/kaggle/input/wine-data/wine.data',header=None)
data.head(3)

0      1     2     3     4    5     6     7     8     9     10    11    12  \
0   1  14.23  1.71  2.43  15.6  127  2.80  3.06  0.28  2.29  5.64  1.04  3.92   
1   1  13.20  1.78  2.14  11.2  100  2.65  2.76  0.26  1.28  4.38  1.05  3.40   
2   1  13.16  2.36  2.67  18.6  101  2.80  3.24  0.30  2.81  5.68  1.03  3.17   

     13  
0  1065  
1  1050  
2  1185

# **Data Exploration**

In [4]:
data.columns  = ['Class_Id'
                 ,'Alcohol'
                 ,'Malic_Acid'
                 ,'Ash'
                 ,'Alcalinity_Ash'
                 ,'Magnesium'
                 ,'Total_Phenols'
                 ,'Flavanoids'
                 ,'NonFlavanoid_Phenols'
                 ,'Proanthocyanins'
                 ,'Color_Intensity'
                 ,'Hue'
                 ,'OD280_OD315_DWines'
                 ,'Proline'
                ]
#data.columns = columns_names

data.head(3)

Class_Id  Alcohol  Malic_Acid   Ash  Alcalinity_Ash  Magnesium  \
0         1    14.23        1.71  2.43            15.6        127   
1         1    13.20        1.78  2.14            11.2        100   
2         1    13.16        2.36  2.67            18.6        101   

   Total_Phenols  Flavanoids  NonFlavanoid_Phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   

   Color_Intensity   Hue  OD280_OD315_DWines  Proline  
0             5.64  1.04                3.92     1065  
1             4.38  1.05                3.40     1050  
2             5.68  1.03                3.17     1185

**Check Missing Values**

In [5]:
data.isnull().sum()

Class_Id                0
Alcohol                 0
Malic_Acid              0
Ash                     0
Alcalinity_Ash          0
Magnesium               0
Total_Phenols           0
Flavanoids              0
NonFlavanoid_Phenols    0
Proanthocyanins         0
Color_Intensity         0
Hue                     0
OD280_OD315_DWines      0
Proline                 0
dtype: int64

In [6]:
data.info() # prints out a basic information about the data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
Class_Id                178 non-null int64
Alcohol                 178 non-null float64
Malic_Acid              178 non-null float64
Ash                     178 non-null float64
Alcalinity_Ash          178 non-null float64
Magnesium               178 non-null int64
Total_Phenols           178 non-null float64
Flavanoids              178 non-null float64
NonFlavanoid_Phenols    178 non-null float64
Proanthocyanins         178 non-null float64
Color_Intensity         178 non-null float64
Hue                     178 non-null float64
OD280_OD315_DWines      178 non-null float64
Proline                 178 non-null int64
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


 **Statistical overview of Data**

In [7]:
# This method prints us some summary statistics for each column in our data.
data.describe()

Class_Id     Alcohol  Malic_Acid         Ash  Alcalinity_Ash  \
count  178.000000  178.000000  178.000000  178.000000      178.000000   
mean     1.938202   13.000618    2.336348    2.366517       19.494944   
std      0.775035    0.811827    1.117146    0.274344        3.339564   
min      1.000000   11.030000    0.740000    1.360000       10.600000   
25%      1.000000   12.362500    1.602500    2.210000       17.200000   
50%      2.000000   13.050000    1.865000    2.360000       19.500000   
75%      3.000000   13.677500    3.082500    2.557500       21.500000   
max      3.000000   14.830000    5.800000    3.230000       30.000000   

        Magnesium  Total_Phenols  Flavanoids  NonFlavanoid_Phenols  \
count  178.000000     178.000000  178.000000            178.000000   
mean    99.741573       2.295112    2.029270              0.361854   
std     14.282484       0.625851    0.998859              0.124453   
min     70.000000       0.980000    0.340000              0.130000   
25%     88.000000       1.742500    1.205000              0.270000   
50%     98.000000       2.355000    2.135000              0.340000   
75%    107.000000       2.800000    2.875000              0.437500   
max    162.000000       3.880000    5.080000              0.660000   

       Proanthocyanins  Color_Intensity         Hue  OD280_OD315_DWines  \
count       178.000000       178.000000  178.000000          178.000000   
mean          1.590899         5.058090    0.957449            2.611685   
std           0.572359         2.318286    0.228572            0.709990   
min           0.410000         1.280000    0.480000            1.270000   
25%           1.250000         3.220000    0.782500            1.937500   
50%           1.555000         4.690000    0.965000            2.780000   
75%           1.950000         6.200000    1.120000            3.170000   
max           3.580000        13.000000    1.710000            4.000000   

           Proline  
count   178.000000  
mean    746.893258  
std     314.907474  
min     278.000000  
25%     500.500000  
50%     673.500000  
75%     985.000000  
max    1680.000000

In [8]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode
#import plotly.express as px
init_notebook_mode(connected=True) # to show plots in notebook
# online plotly
from plotly.offline import plot, iplot
#plotly.tools.set_credentials_file(username='XXXXXXXXXXXXXX', api_key='XXXXXXXXXXXXXX')

# do not show any warnings
import warnings
warnings.filterwarnings('ignore')

colors = plotly.colors.DEFAULT_PLOTLY_COLORS
class_dict = {1: "class 1", 2: "class 2",3:"class 3"}

**Class Distribution**

In [9]:
y=data["Class_Id"].value_counts()
data_bar = [go.Bar(x=[class_dict[x] for x in y.index], 
                y=y.values,
                marker=dict(color=colors[:len(y.index)])
               )]
layout = go.Layout(
    title='Class Distribution',
    autosize=False,
    width=400,
    height=400,
    yaxis=dict(
        title='#Samples',
    ),
)
fig = go.Figure(data=data_bar, layout= layout)
#fig.show()
iplot(fig, filename='basic-bar3')

**Percentage Class Distribution**

In [10]:
import plotly.graph_objects as go

data_3=[go.Pie(
    labels=[class_dict[x] for x in y.index],
    values=y.values,
    marker=dict(colors=colors[:len(y.index)])
    
)]

layout_percent = go.Layout(
    title='Percentage class Distribution',
    autosize=False,
    width=400,
    height=400,
)
fig=go.Figure(data=data_3,layout=layout_percent)
fig.show()
#iplot(fig, filename='new')

**Box plot for each features**

In [11]:
pre_data = data.copy()
pre_data.head(3)

Class_Id  Alcohol  Malic_Acid   Ash  Alcalinity_Ash  Magnesium  \
0         1    14.23        1.71  2.43            15.6        127   
1         1    13.20        1.78  2.14            11.2        100   
2         1    13.16        2.36  2.67            18.6        101   

   Total_Phenols  Flavanoids  NonFlavanoid_Phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   

   Color_Intensity   Hue  OD280_OD315_DWines  Proline  
0             5.64  1.04                3.92     1065  
1             4.38  1.05                3.40     1050  
2             5.68  1.03                3.17     1185

**Box Plot**

In [12]:
#class_1 = data["Class_Id"].value_counts()
class_1 = pre_data[pre_data["Class_Id"] == 1]
class_2 = pre_data[pre_data["Class_Id"] == 2]
class_3 = pre_data[pre_data["Class_Id"] == 3]
def create_box_class_1_trace(col, visible=False):
    return go.Box(
        y=class_1[col],
        name='class 1',
        marker = dict(color = colors[1]),
        visible=visible,
    )

def create_box_class_2_trace(col, visible=False):
    return go.Box(
        y=class_2[col],
        name='class 2',
        marker = dict(color = colors[2]),
        visible = visible,
    )

def create_box_class_3_trace(col, visible=False):
    return go.Box(
        y=class_3[col],
        name='class 3',
        marker = dict(color = colors[3]),
        visible = visible,
    )
features_not_for_hist = ["Class_Id"]
features_for_hist = [x for x in pre_data.columns if x not in features_not_for_hist]
# remove features with too less distinct values (e.g. binary features), because boxplot does not make any sense for them
features_for_box = [col for col in features_for_hist if len(class_1[col].unique())>5 or len(class_2[col].unique())>5 or len(class_1[col].unique())>5]
#features_for_box = [col for col in features_for_hist]

active_idx = 0
box_traces_class_1 = [(create_box_class_1_trace(col) if i != active_idx else create_box_class_1_trace(col, visible=True)) for i, col in enumerate(features_for_box)]
box_traces_class_2 = [(create_box_class_2_trace(col) if i != active_idx else create_box_class_2_trace(col, visible=True)) for i, col in enumerate(features_for_box)]
box_traces_class_3 = [(create_box_class_3_trace(col) if i != active_idx else create_box_class_3_trace(col, visible=True)) for i, col in enumerate(features_for_box)]

data_box = box_traces_class_1 + box_traces_class_2 + box_traces_class_3

n_features = len(features_for_box)
steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data_box)],
        label = features_for_box[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data_box, layout=layout)
#fig.show()
iplot(fig, filename='box_slider')

 **Correlation Between features**

**Heatmap**
To visualize these correlations we use a heatmap plot, in which high correlations are coloured more to the yellow and lower ones more to the voilet

In [13]:
corr = pre_data.corr()
trace = go.Heatmap(z=corr.values.tolist(), x=corr.columns, y=corr.columns )
data_heatmap=[trace]
layout = go.Layout(
    title='Heatmap of pairwise correlation of the columns',
    autosize=False,
    width=850,
    height=700,
    yaxis=go.layout.YAxis(automargin=True),
    xaxis=dict(tickangle=40),
    margin=go.layout.Margin(l=0, r=200, b=200, t=80),

)


fig = go.Figure(data=data_heatmap, layout=layout)
iplot(fig, filename='labelled-heatmap1')

Dendogram

In [14]:
from scipy.cluster import hierarchy as hc
X = np.random.rand(10, 10)
names = pre_data.columns
inverse_correlation = 1 - abs(pre_data.corr())
fig = ff.create_dendrogram(inverse_correlation.values, orientation='left', labels=names, colorscale=colors, linkagefun=lambda x: hc.linkage(x, 'average'))
fig['layout'].update(dict(
    title="Dendrogram of the features according to correlation",
    width=800, 
    height=600,
    margin=go.layout.Margin(l=180, r=50),
    xaxis=dict(
        title='distance',
    ),
    yaxis=dict(
        title='features',
        automargin=True,
    ),
))
iplot(fig, filename='dendrogram_corr_clustering')

In [15]:
import heapq

print('Absolute overall correlations')
print('-' * 30)
corr_abs_sum = corr[corr.columns].abs().sum()
print(corr_abs_sum, '\n')

print('Weakest correlations')
print('-' * 20)
print(corr_abs_sum.nsmallest(3),'\n')

print('Heighest correlations')
print('-' * 20)
print(corr_abs_sum.nlargest(4))

Absolute overall correlations
------------------------------
Class_Id                7.402563
Alcohol                 4.367915
Malic_Acid              4.670193
Ash                     3.156218
Alcalinity_Ash          4.885079
Magnesium               3.522095
Total_Phenols           6.621719
Flavanoids              7.209950
NonFlavanoid_Phenols    5.312493
Proanthocyanins         5.101221
Color_Intensity         4.197137
Hue                     5.513241
OD280_OD315_DWines      6.392491
Proline                 6.026176
dtype: float64 

Weakest correlations
--------------------
Ash                3.156218
Magnesium          3.522095
Color_Intensity    4.197137
dtype: float64 

Heighest correlations
--------------------
Class_Id              7.402563
Flavanoids            7.209950
Total_Phenols         6.621719
OD280_OD315_DWines    6.392491
dtype: float64


# **Feature Extraction:-**


In [16]:
# splitting the dataset into feature vectors and the target variable
data_y = pre_data["Class_Id"]
data_X = pre_data.drop(["Class_Id"], axis=1)
data_X.head(3)

Alcohol  Malic_Acid   Ash  Alcalinity_Ash  Magnesium  Total_Phenols  \
0    14.23        1.71  2.43            15.6        127           2.80   
1    13.20        1.78  2.14            11.2        100           2.65   
2    13.16        2.36  2.67            18.6        101           2.80   

   Flavanoids  NonFlavanoid_Phenols  Proanthocyanins  Color_Intensity   Hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   

   OD280_OD315_DWines  Proline  
0                3.92     1065  
1                3.40     1050  
2                3.17     1185

In [17]:
# normalize the dataset (note: for decision tree/random forest it would not be needed)
data_X_normed = (data_X - data_X.mean()) / data_X.std()
data_X_normed.head(3)

Alcohol  Malic_Acid       Ash  Alcalinity_Ash  Magnesium  Total_Phenols  \
0  1.514341   -0.560668  0.231400       -1.166303   1.908522       0.806722   
1  0.245597   -0.498009 -0.825667       -2.483841   0.018094       0.567048   
2  0.196325    0.021172  1.106214       -0.267982   0.088110       0.806722   

   Flavanoids  NonFlavanoid_Phenols  Proanthocyanins  Color_Intensity  \
0    1.031908             -0.657708         1.221438         0.251009   
1    0.731565             -0.818411        -0.543189        -0.292496   
2    1.212114             -0.497005         2.129959         0.268263   

        Hue  OD280_OD315_DWines   Proline  
0  0.361158            1.842721  1.010159  
1  0.404908            1.110317  0.962526  
2  0.317409            0.786369  1.391224

We apply PCA to the dataset in order to reduce the number of features

In [18]:
# calculate the principal components
pca = PCA(random_state=SEED)
data_X_pca = pca.fit_transform(data_X_normed)

In [19]:
tot = sum(pca.explained_variance_) # total explained variance of all principal components
var_exp = [(i / tot) * 100 for i in sorted(pca.explained_variance_, reverse=True)] # individual explained variance
cum_var_exp = np.cumsum(var_exp) # cumulative explained variance


In the following we plot the Scree Plot to determine how many components we use.

In [20]:
trace_cum_var_exp = go.Bar(
    x=list(range(1, len(cum_var_exp) + 1)), 
    y=var_exp,
    name="individual explained variance",
)
trace_ind_var_exp = go.Scatter(
    x=list(range(1, len(cum_var_exp) + 1)),
    y=cum_var_exp,
    mode='lines+markers',
    name="cumulative explained variance",
    line=dict(
        shape='hv',
    ))
data_pca = [trace_cum_var_exp, trace_ind_var_exp]
layout = go.Layout(
    title='Individual and Cumulative Explained Variance',
    autosize=True,
    yaxis=dict(
        title='percentage of explained variance',
    ),
    xaxis=dict(
        title="principal components",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data_pca, layout=layout)
iplot(fig, filename='basic-bar')

In [21]:
n_components = 7
data_X_reduced = np.dot(data_X_normed.values, pca.components_[:n_components,:].T)
data_X_reduced = pd.DataFrame(data_X_reduced, columns=["PC#%d" % (x + 1) for x in range(n_components)])
data_X_reduced.head(3)

PC#1      PC#2      PC#3      PC#4      PC#5      PC#6      PC#7
0  3.307421 -1.439402 -0.165273 -0.215025  0.691093 -0.223250  0.594749
1  2.203250  0.332455 -2.020757 -0.290539 -0.256930 -0.924512  0.053624
2  2.509661 -1.028251  0.980054  0.722863 -0.250327  0.547731  0.423012

 **Comparison of differently preprocessed datasets for classification**
 
In this chapter we apply one classifier to the different version of the dataset. The Random Forest classifier is used because it is considered as a great baseline model for most applications. It is described in more detail in the next chapter. The following different versions of the dataset are investigated:

* full dataset
* dataset with variables reduced by the clustering according to the correlation
* dataset reduced by considering the first seven principal components after applying PCA

Furthermore, we apply two different methods to deal with the unbalanced target variable. First, we try adjusting the weights for the penalization when the classifier makes a mistake in the training and then we try to oversample the data using the Synthetic Minority Over-sampling Technique.

For the evaluation of the classifiers on the different datasets, a hold-out test set is used, which has 20% of all the data. To account for the class imbalance of our target variable, we use the f1-score as our main evaluation metric. We define:

TP = #samples for which the prediction is positive and the true label is positive
FP = #samples for which the prediction is positive but the true label is negative
TN = #samples for which the prediction is negative and the true label is negative
FN = #samples for which the prediction is negative but the true label is positive
Then we define the following:

$\text{precision} = \frac{TP}{TP + FP} \;\;\; \text{and} \;\;\; \text{recall} = \frac{TP}{TP + FN}$

Then the f1-score is given by the following equation:

$F_{1}=2\,\frac{\text{precision}\, \times \,\text{recall}}{\text{precision} \, +\, \text{recall}}$

Every classifier has a set of hyperparameters, which can be tuned by training the classifier with different values for these hyperparameters and selecting the classifier with the best score. In order to estimate the performance of a classifier in a more reliable way, k-fold cross validation (CV) is used. In k-fold CV, the training set is divided into a k subsets. Then we train k times our classifier on different unions of k-1 subsets and calculate its score on the subset which was not used for training. Then the final score is calculated by averaging the score of each iteration. In detail, let $C_1, C_2, ... C_k$ be the indices of the samples in each of the $K$ parts of the dataset and let $n_k$ be the number of observations in part $k$. Then the score from the cross validation is computed as follows:

$\text{Score}_{CV(K)} = \sum_{k=1}^{K} \frac{n_k}{n}\text{Score}_k$.

In our hyper parameter tuning the Score is the f1-Score defined above.

To do this analysis, we use the sklearn.model_selection.GridSearchCV object, to which we pass a classifier, a dictionary of hyperparameters with values and a $k$-fold object. For a good trade-off between runtime and accuracy of the score we choose $k=5$, so the classifiers are trained on 80% of the train data in each iteration.

 **GridsearchCV for Hyperparameter Tuning**

In [22]:
# prints the best grid search scores along with their parameters.
def print_best_grid_search_scores_with_params(grid_search, n=5):
    if not hasattr(grid_search, 'best_score_'):
        raise KeyError('grid_search is not fitted.')
    print("Best grid scores on validation set:")
    indexes = np.argsort(grid_search.cv_results_['mean_test_score'])[::-1][:n]
    means = grid_search.cv_results_['mean_test_score'][indexes]
    stds = grid_search.cv_results_['std_test_score'][indexes]
    params = np.array(grid_search.cv_results_['params'])[indexes]
    for mean, std, params in zip(means, stds, params):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [23]:
def do_gridsearch_with_cv(clf, params, X_train, y_train, cv, smote=None):

    if smote is None:
        pipeline = Pipeline([('clf', clf)])
    else:
        pipeline = Pipeline([('sm', sm), ('clf', clf)])
    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average='weighted') 
    gs = GridSearchCV(pipeline, params, cv=kf, n_jobs=-1, scoring=scorer, return_train_score=True)
    gs.fit(X_train, y_train)
    return gs

def score_on_test_set(clfs, datasets):
    scores = []
    for c, (X_test, y_test) in zip(clfs, datasets):
        scores.append(c.score(X_test, y_test))
    return scores

** Train Test Split**

In [24]:
#features closely related
correlated_features = ["Flavanoids","Total_Phenols","OD280_OD315_DWines"]

In [25]:
# split data into train and test set in proportion 4:1 for all differntly preprocessed datasets
#full train dataset
X_train, X_test, y_train, y_test = train_test_split(data_X_normed, data_y, test_size=0.2, random_state=SEED)
#removing the most correlated features
cols_without_duplicate = [x for x in data_X_normed.columns if x not in correlated_features]
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(data_X_normed[cols_without_duplicate], data_y, test_size=0.2, random_state=SEED)
#using pca
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(data_X_reduced, data_y, test_size=0.2, random_state=SEED)


In [26]:
print("Full train dataset:", X_train.shape)
print("Train dataset with reduced features", X_train_red.shape)
print("Train dataset using the first 7 Principal Components", X_train_pca.shape)

Full train dataset: (142, 13)
Train dataset with reduced features (142, 10)
Train dataset using the first 7 Principal Components (142, 7)


In [27]:
sm = SMOTE(random_state=SEED)
kf = StratifiedKFold(n_splits=5, random_state=SEED)
clf_rf = RandomForestClassifier(random_state=SEED)
clf_balanced = RandomForestClassifier(random_state=SEED, class_weight="balanced")

**Without additional balancing techniques**

We train the Random Forest on the full dataset, on the dataset with reduced features and on the dataset transformed using the first seven principal components.

In [28]:
%%time
import sklearn
gs_full = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=None)
gs_red = do_gridsearch_with_cv(clf_rf,RANDOM_FOREST_PARAMS, X_train_red, y_train_red, kf, smote=None)
gs_pca = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_pca, y_train_pca, kf, smote=None)
gss_raw = [gs_full, gs_red, gs_pca]

CPU times: user 3.98 s, sys: 148 ms, total: 4.13 s
Wall time: 2min 51s


In [29]:
test_results_raw = score_on_test_set(gss_raw, [(X_test, y_test),(X_test_red, y_test_red), (X_test_pca, y_test_pca)])

**Using class weights in the loss function**

The sklearn library offers for all parametric classifiers a parameter class_weight, which can be set to "balanced". Then, mistakes are weighted inversely proportional to the class frequencies. This means that mistakes for the minority class are penalized more than mistakes made for the majority class.

In [30]:
%%time
gs_full_balanced = do_gridsearch_with_cv(clf_balanced, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=None)
gs_red_balanced = do_gridsearch_with_cv(clf_balanced, RANDOM_FOREST_PARAMS, X_train_red, y_train_red, kf, smote=None)
gs_pca_balanced = do_gridsearch_with_cv(clf_balanced, RANDOM_FOREST_PARAMS, X_train_pca, y_train_pca, kf, smote=None)
gss_balanced_weights = [gs_full_balanced,gs_red_balanced, gs_pca_balanced]

CPU times: user 3.94 s, sys: 61.1 ms, total: 4 s
Wall time: 2min 45s


In [31]:
test_results_balanced_weights = score_on_test_set(gss_balanced_weights, [(X_test, y_test),(X_test_red, y_test_red), (X_test_pca, y_test_pca)])

**Using Synthetic Minority Over-sampling Technique¶**

The Synthetic Minority Over-sampling Technique (SMOTE) algorithm applies KNN approach where it selects one of the k nearest neighbors and computes the vector between the original point and the selected neighbor. The difference is multiplied by random number between (0, 1) and it is added back to original point to obtain the new synthetic point. Geometrically, the synthetic point is somewhere on the line between the original point and its neighbor.

In the following, we use the SMOTE implementation SMOTE of the imblearn.over_sampling library. Furthermore, we create a Pipeline of applying Smote and then training the classifier, so that it is executed in every fold of x-fold cross validation.

In [32]:
%%time
gs_full_smote = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=sm)
gs_red_smote = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_red, y_train_red, kf, smote=sm)
gs_pca_smote = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_pca, y_train_pca, kf, smote=sm)
gss_smote = [gs_full_smote,gs_red_smote, gs_pca_smote]

CPU times: user 3.71 s, sys: 59.1 ms, total: 3.77 s
Wall time: 2min 46s


In [33]:

test_results_smote = score_on_test_set(gss_smote, [(X_test, y_test),(X_test_red,y_test_red), (X_test_pca, y_test_pca)])

**Comparison of differently processed Dataset**

In [34]:
dataset_strings = ["full dataset","dataset with Reduced","dataset with first 7 principal components"]
method_strings = ["without any balancing", "using balanced class weights", "using SMOTE"]

result_strings = dict()
for ms, results in zip(method_strings, [test_results_raw, test_results_balanced_weights, test_results_smote]):
    for ds, res in zip(dataset_strings, results):
        string = "%.3f" % res + "     " + ds + " " + ms
        result_strings[string] = res
        2
result_strings = sorted(result_strings.items(), key=lambda kv: kv[1], reverse=True)
print("F1 score  dataset and method")
print("-"* 30)
for k,_ in result_strings:
    print(k)

F1 score  dataset and method
------------------------------
0.946     full dataset without any balancing
0.946     dataset with first 7 principal components without any balancing
0.946     full dataset using balanced class weights
0.946     full dataset using SMOTE
0.944     dataset with Reduced without any balancing
0.944     dataset with Reduced using balanced class weights
0.944     dataset with Reduced using SMOTE
0.944     dataset with first 7 principal components using balanced class weights
0.944     dataset with first 7 principal components using SMOTE


On the basis of f1 score for the datasets we can say that using full dataset in all the three techniques gives the best results. As a conclusion we will use the full dataset and apply SMOTE. 

In [35]:
def get_color_with_opacity(color, opacity):
    return "rgba(" + color[4:-1] + ", %.2f)" % opacity

# partially based on https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scorer, random_state=SEED)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    trace1 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean - train_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[0], 0.4),
        ),
    )
    trace2 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean + train_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[0], 0.4),
        ),
    )
    trace3 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean, 
        showlegend=True,
        name="Train score",
        line = dict(
            color = colors[0],
        ),
    )
    
    trace4 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean - test_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[1], 0.4),
        ),
    )
    trace5 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean + test_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[1], 0.4),
        ),
    )
    trace6 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean, 
        showlegend=True,
        name="Test score",
        line = dict(
            color = colors[1],
        ),
    )
    
    data_scatter = [trace1, trace2, trace3, trace4,trace5, trace6]
    layout = go.Layout(
        title=title,
        autosize=True,
        yaxis=dict(
            title='F1 Score',
        ),
        xaxis=dict(
            title="#Training samples",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = go.Figure(data=data_scatter, layout=layout)
    return iplot(fig, filename=title)

In [36]:
def plot_feature_importance(feature_importance, title):
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = colors[0]),
        name='feature importance'
    )
    data = [trace1]
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='feature importance',
            automargin=True,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)

# **Classification**


# **Logistic Regression**

We define $x_i$ as the $n$-dimensional feature vector of a given sample and $\beta_{0},\,\boldsymbol{\beta} = (\beta_{1}, ..., \beta_{n})^T$ as the model parameters. Then the logistic regression model is defined as:

$P(Y=1 \vert x_i)= \frac{\text{exp}(\beta_{0} + x_i^T\boldsymbol{\beta} )}{1+\text{exp}(\beta_{0} + x_i^T\boldsymbol{\beta} )} = \frac{1}{1+\text{exp}(-(\beta_{0} + x_i^T\boldsymbol{\beta} ))}$

The hyperparameters of a logistic regression include the following ones, which can be passed to the LogisticRegression of sklearn.linear_model:

penalty: the norm used for penalization (default='l2')
C: the inverse of the regularization strength (default=1.0)

In [37]:
%%time
clf_lr = LogisticRegression(random_state=SEED)
gs_lr = do_gridsearch_with_cv(clf_lr, LOGISTIC_REGRESSION_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 156 ms, sys: 5.9 ms, total: 162 ms
Wall time: 294 ms


In [38]:
print_best_grid_search_scores_with_params(gs_lr)

Best grid scores on validation set:
0.993 (+/-0.028) for {'clf__C': 10, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
0.993 (+/-0.028) for {'clf__C': 10, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
0.993 (+/-0.029) for {'clf__C': 1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
0.986 (+/-0.034) for {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
0.979 (+/-0.035) for {'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}


In [39]:
gs_lr_score = gs_lr.score(X_test, y_test)
y_pred_lr = gs_lr.predict(X_test)
cm_lr = confusion_matrix(y_test, y_pred_lr)
cm_lr = cm_lr.astype('float') / cm_lr.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In [40]:
cm_df = pd.DataFrame(cm_lr.round(3), index=["actual class 1", "actual class 2","actual class 3"], columns=["predicted class 1", "predicted class 2", "predicted class 3"])
cm_df

predicted class 1  predicted class 2  predicted class 3
actual class 1                1.0              0.000              0.000
actual class 2                0.0              0.833              0.167
actual class 3                0.0              0.000              1.000

In [41]:
plot_learning_curve(gs_lr.best_estimator_, "Learning Curve of Logistic Regression", X_train, y_train, cv=5)


# **K-Nearest Neighbors**
The K-Nearest Neighbors algorithm (KNN) is a non-parametric method, which considers the K closest training examples to the point of interest for predicting its class. This is done by a simple majority vote over the K closest points.

The hyperparameters of KNN include the following ones, which can be passed to the KNeighborsClassifier of sklearn.neighbors:

n_neighbors: corresponds to K, the number of nearest neighbors considered for the prediction (default=5)
weights:
if uniform, then all neighbors have the same weight for the voting (default)
if distance, then the votes of the neighbors are weighted by the inverse of the distance for the voting
p: the power parameter for the Minkowski metric (default=2)

In [42]:
%%time
clf_knn = KNeighborsClassifier()
gs_knn = do_gridsearch_with_cv(clf_knn, KNN_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 629 ms, sys: 10.9 ms, total: 640 ms
Wall time: 2.45 s


In [43]:
print_best_grid_search_scores_with_params(gs_knn)

Best grid scores on validation set:
0.979 (+/-0.035) for {'clf__n_neighbors': 25, 'clf__p': 2, 'clf__weights': 'uniform'}
0.979 (+/-0.035) for {'clf__n_neighbors': 25, 'clf__p': 2, 'clf__weights': 'distance'}
0.965 (+/-0.044) for {'clf__n_neighbors': 45, 'clf__p': 1, 'clf__weights': 'uniform'}
0.965 (+/-0.044) for {'clf__n_neighbors': 35, 'clf__p': 1, 'clf__weights': 'distance'}
0.965 (+/-0.044) for {'clf__n_neighbors': 25, 'clf__p': 1, 'clf__weights': 'distance'}


In [44]:

gs_knn_score = gs_knn.score(X_test, y_test)
y_pred_knn = gs_knn.predict(X_test)
cm_knn = confusion_matrix(y_test, y_pred_knn)
cm_knn = cm_knn.astype('float') / cm_knn.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In [45]:
cm_df = pd.DataFrame(cm_knn.round(3), index=["actual class 1", "actual class 2","actual class 3"], columns=["predicted class 1", "predicted class 2", "predicted class 3"])
cm_df

predicted class 1  predicted class 2  predicted class 3
actual class 1              1.000              0.000              0.000
actual class 2              0.056              0.722              0.222
actual class 3              0.000              0.000              1.000

In [46]:
plot_learning_curve(gs_knn.best_estimator_, "Learning Curve of KNN", X_train, y_train, cv=5)

# **Support-Vector Machine**
A linear Support-Vector Machine (SVM) finds the optimal hyperplane between the points of two classes such that the distance of the nearest points to the decision boundary is maximized. This distance is called margin.

If the data set is not linearly separable, we can map the samples ${\bf x}$ into a feature space of higher dimensions: ${\bf x} \longrightarrow \phi({\bf x})$ in which the classes can be linearly separated. This results in a non-linear decision boundary in the original dimensions.

As the vectors ${\bf x}_i$ appear only in inner products in both the decision function and the learning law, the mapping function $\phi({\bf x})$ does not need to be explicitly specified. Instead, we define a so-called kernel function:

$ K({\bf x}_1,{\bf x}_2)=\phi({\bf x}_1)^T\phi({\bf x}_2)$.

In the gridsearch we consider the following two kernels:

linear kernel: $ K({\bf x}_1,{\bf x}_2) = {\bf x}_1 \cdot {\bf x}_2$
radial basis function: $ K({\bf x}_1,{\bf x}_2) = exp(-\gamma({\Vert {\bf x}_1 - {\bf x}_2 \Vert}^2))$
The hyperparameters of a SVM include the following ones, which can be passed to the SVC of sklearn.svm:

C: the inverse of the regularization strength (default=1.0)
kernel: the kernel used (default='rbf')
gamma: The higher the gamma value it tries to exactly fit the training data set (default='auto_deprecated'

In [47]:
%%time
clf_svm = svm.SVC(random_state=SEED, probability=True)
gs_svm = do_gridsearch_with_cv(clf_svm, SVM_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 457 ms, sys: 8.51 ms, total: 466 ms
Wall time: 1.41 s


In [48]:
print_best_grid_search_scores_with_params(gs_svm)

Best grid scores on validation set:
1.000 (+/-0.000) for {'clf__C': 1, 'clf__kernel': 'linear'}
1.000 (+/-0.000) for {'clf__C': 100, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 100, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 10, 'clf__kernel': 'linear'}
1.000 (+/-0.000) for {'clf__C': 10, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}


In [49]:
gs_svm_score = gs_svm.score(X_test, y_test)
y_pred_svm = gs_svm.predict(X_test)
cm_svm = confusion_matrix(y_test, y_pred_svm)
cm_svm = cm_svm.astype('float') / cm_svm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix


In the following, the normalized confusion matrix is shown:

In [50]:
cm_df = pd.DataFrame(cm_svm.round(3), index=["actual class 1", "actual class 2","actual class 3"], columns=["predicted class 1", "predicted class 2", "predicted class 3"])
cm_df

predicted class 1  predicted class 2  predicted class 3
actual class 1                1.0              0.000              0.000
actual class 2                0.0              0.833              0.167
actual class 3                0.0              0.000              1.000

The confusion matrix shows that the SVM classifier has a clear bias towards predicting class 3.

In [51]:
plot_learning_curve(gs_svm.best_estimator_, "Learning Curve of SVM", X_train, y_train, cv=5)

# Decision Tree
A decision tree for classification consists of several splits, which determine for a input sample, the predicted class, which is a leaf node in the tree. The construction of the decision trees is done with a greedy algorithm, because the theoretical minimum of function exists but it is NP-hard to determine it, because number of partitions has a factorial growth. Specifically, a greedy top-down approach is used which chooses a variable at each step that best splits the set of items. For measuring the "best" different metrics can be used, which generally measure the homogeneity of the target variable within the subsets. For this analysis we consider the following two metrics:

Gini impurity: Let $j$ be the number of classes and $p_i$ the fraction of items of class $i$ in a subset $p$, for $i \in \{1,2,..., j\}$. Then the gini impurity is defined as follows: $\;I_G(p) = 1- \sum_{i=1}^j {p_i}^2$.

Information gain: It measures the reduction in entropy when applying the split. The entropy is defined as $H(t) = - \sum_{i=1}^j p_i\, \text{log}_2\,p_i$. Then we define the information gain to split $n$ samples in parent node $p$ into $k$ partitions, where $n_i$ is the number of samples in partition $i$ as $IG = H(p) - \sum_{i = 1}^k \frac{n_i}{n} H(i)$.

The hyperparameters of a Decision Tree include the following ones, which can be passed to the DecisionTreeClassifier of sklearn.tree:

criterion: the criterion which decides the feature and the value at the split (default='gini')
max_depth: the maximum depth of each tree (default=None)
min_samples_split: the minimum number of samples in a node to be considered for further splitting (default=2)

In [52]:
%%time
clf_dt = DecisionTreeClassifier(random_state=SEED)
gs_dt = do_gridsearch_with_cv(clf_dt, DECISION_TREE_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 294 ms, sys: 3.98 ms, total: 298 ms
Wall time: 718 ms


In [53]:
print_best_grid_search_scores_with_params(gs_dt)

Best grid scores on validation set:
0.916 (+/-0.085) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.916 (+/-0.085) for {'clf__criterion': 'entropy', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.916 (+/-0.085) for {'clf__criterion': 'entropy', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.901 (+/-0.136) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.901 (+/-0.136) for {'clf__criterion': 'gini', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}


In [54]:
gs_dt_score = gs_dt.score(X_test, y_test)
y_pred_dt = gs_dt.predict(X_test)
cm_dt = confusion_matrix(y_test, y_pred_dt)
cm_dt = cm_dt.astype('float') / cm_dt.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In [55]:
cm_df = pd.DataFrame(cm_dt.round(3), index=["actual class 1", "actual class 2","actual class 3"], columns=["predicted class 1", "predicted class 2", "predicted class 3"])
cm_df

predicted class 1  predicted class 2  predicted class 3
actual class 1              1.000              0.000              0.000
actual class 2              0.278              0.444              0.278
actual class 3              0.000              0.000              1.000

In [56]:
feature_importance = np.array(sorted(zip(X_train.columns, gs_dt.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance, "Feature importance in the decision tree")

In [57]:

plot_learning_curve(gs_dt.best_estimator_, "Learning Curve of the Decision Tree", X_train, y_train, cv=5)


# Random Forest
A random forest is an ensemble model that fits a number of decision tree classifiers on various sub-samples of the dataset which are created by the use of bootstrapping. In the inference stage it uses a majority vote over all trees to obtain the prediction. This improves the predictive accuracy and controls over-fitting.

The hyperparameters of a random forest include the following ones, which can be passed to the RandomForestClassifier of sklearn.ensemble:

n_estimators: the number of trees
criterion: the criterion which decides the feature and the value at the split (default='gini')
max_depth: the maximum depth of each tree (default=None)
min_samples_split: the minimum number of samples in a node to be considered for further splitting (default=2)
max_features: the number of features which are considered for a split (default='sqrt')

In [58]:
%%time
clf_rf = RandomForestClassifier(random_state=SEED)
gs_rf = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 1.07 s, sys: 21.8 ms, total: 1.09 s
Wall time: 55 s


In [59]:
print_best_grid_search_scores_with_params(gs_rf)

Best grid scores on validation set:
0.986 (+/-0.035) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__n_estimators': 1000}
0.986 (+/-0.035) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
0.986 (+/-0.035) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__n_estimators': 300}
0.986 (+/-0.035) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
0.986 (+/-0.035) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__n_estimators': 1000}


In [60]:
gs_rf_score = gs_rf.score(X_test, y_test)
y_pred_rf = gs_rf.predict(X_test)
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_rf = cm_rf.astype('float') / cm_rf.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In [61]:
cm_df = pd.DataFrame(cm_rf.round(3), index=["actual class 1", "actual class 2","actual class 3"], columns=["predicted class 1", "predicted class 2", "predicted class 3"])
cm_df

predicted class 1  predicted class 2  predicted class 3
actual class 1                1.0              0.000              0.000
actual class 2                0.0              0.889              0.111
actual class 3                0.0              0.000              1.000

In [62]:
feature_importance_rf = np.array(sorted(zip(X_train.columns, gs_rf.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance_rf, "Feature importance in the Random Forest")

In [63]:
plot_learning_curve(gs_dt.best_estimator_, "Learning Curve of the Random Forest", X_train, y_train, cv=5)

# **Conclusion¶**
To conclude the performance of the different classification model, we consider several evalutation metric in addition to the in chapter 4 defined precision, recall and f1-score. Further, let TP, FP, TN, FN be defined as in chapter 4.

Accuracy
The accuracy is the percentage of samples classified correctly: $\text{accuracy} = \frac{TP + TN}{TP + FP + TN + FN}$.

Area Under the Receiver Operating Characteristic curve (AUC)
To introduce this concept, we define the following two metrics:

True positive rate (TPR): this is the same as the recall: $FPR = \text{recall} = \frac{TP}{FN + TP}$
False positive rate (FPR): this corresponds to the proportion of negative data points that are mistakenly considered as positive, with respect to all negative data points: $FPR = \frac{FP}{TN + FP}$
To plot the Receiver Operating Characteristic (ROC) curve we choose a number of different classification thresholds and compute the TPR and the FPR. So the curve shows the trade-off between these two. To combine the TPR and the FPR into one evaluation metric the area under the ROC curve (AUC) is computed.

The following plot shows the ROC curves of the classifiers trained in the previous chapters:

In [64]:
# code partially from https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
def plot_roc_curve(classifiers, legend, title, X_test, y_test):
    trace1 = go.Scatter(
        x=[0, 1], 
        y=[0, 1], 
        showlegend=False,
        mode="lines",
        name="",
        line = dict(
            color = colors[0],
        ),
    )
    
    data = [trace1]
    aucs = []
    #accuracys = []
    for clf, string, c in zip(classifiers, legend, colors[1:]):
        y_test_roc = np.array([([1,2,3] if y else [3,2,1]) for y in y_test])
        y_score = clf.predict_proba(X_test)
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(2):
            fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_score[:, i],pos_label=2)
            roc_auc[i] = auc(fpr[i], tpr[i])
            #roc_auc[i] = accuracy_score(fpr[i],tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_score.ravel(),pos_label=2)
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        #roc_auc["micro"] = accuracy_score(fpr["micro"], tpr["micro"])
        
        aucs.append(roc_auc['micro'])
        
       # accuracys.append(roc_auc['micro'])

        trace = go.Scatter(
            x=fpr['micro'], 
            y=tpr['micro'], 
            showlegend=True,
            mode="lines",
            name=string + " (area = %0.2f)" % roc_auc['micro'],
            hoverlabel = dict(
                namelength=30
            ),
            line = dict(
                color = c,
            ),
        )
        data.append(trace)

    layout = go.Layout(
        title=title,
        autosize=False,
        width=550,
        height=550,
        yaxis=dict(
            title='True Positive Rate',
        ),
        xaxis=dict(
            title="False Positive Rate",
        ),
        legend=dict(
            x=0.4,
            y=0.06,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return aucs, iplot(fig, filename=title)

In [65]:
classifiers = [gs_lr, gs_knn, gs_svm, gs_dt, gs_rf]
classifier_names = ["Logistic Regression", "KNN", "SVM", "Decision Tree", "Random Forest"]
auc_scores, roc_plot = plot_roc_curve(classifiers, classifier_names, "ROC curve", X_test, y_test)
roc_plot

In [66]:
accs = []
recalls = []
precision = []
results_table = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1", "auc"])
for (i, clf), name, auc in zip(enumerate(classifiers), classifier_names, auc_scores):
    y_pred = clf.predict(X_test)
    row = []
    row.append(accuracy_score(y_test, y_pred))
    row.append(precision_score(y_test, y_pred,average='weighted'))
    row.append(recall_score(y_test, y_pred,average='weighted'))
    row.append(f1_score(y_test, y_pred,average='weighted'))
    row.append(auc)
    row = ["%.3f" % r for r in row]
    results_table.loc[name] = row

In [67]:
results_table

accuracy precision recall     f1    auc
Logistic Regression    0.917     0.939  0.917  0.919  0.585
KNN                    0.861     0.901  0.861  0.862  0.516
SVM                    0.917     0.939  0.917  0.919  0.627
Decision Tree          0.722     0.822  0.722  0.699  0.427
Random Forest          0.944     0.956  0.944  0.946  0.526